# REMIND Electricity Sector Biosphere Flows

Calculate all biosphere flow coefficients for the REMIND electricity sector. **Values are given per kWh**.

In [102]:
import helpers.eimod as eimod

In [2]:
import helpers.ei2rmnd as ei2rmnd

In [3]:
%run initialize_notebook.ipynb

In [4]:
databases

Databases dictionary with 9 object(s):
	Carma CCS
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050

## Bioflows without double counting

Calculate impact for an electricity generating technology in the REMIND power sector excluding *other* activities that are part of the electricity sector to avoid double counting.

In [100]:
def rmnd_bioflows_without_double_counting(region, scenario="BAU", year=2030, tech_primers={}):
    """ Extract bioflows for the REMIND electricity sector in a REMIND region.
    
    In this version, for every REMIND technology, we exclude all other technologies
    when performing the LCA to avoid double counting.
        
    A dict with primers {x: y} can be given to specify ecoinvent technology y to represent
    REMIND technology x.
    """
    eidb_name = ei2rmnd.get_REMIND_database_name(scenario, year)
    eidb = Database(eidb_name)

    result = {}
    bio_names = []
    regions = ei2rmnd.ei_locations_in_remind_region(region)
    print("Found following regions for {}: {}".format(region, regions))
    
    # flatten activity list
    all_actvt_names = [act for techs in eimod.available_electricity_generating_technologies.values() for act in techs]
    all_actvts = [act for act in eidb if act["name"] in all_actvt_names] 
    
    for act_name in all_actvt_names:
        print("Processing activity {}".format(act_name))

        demand = 1.
        # CHP plants are a problem, not only because they provide energy in megajoule units
        # find out which part is electric energy
        if "heat and power co-generation" in act_name:
            demand = 1./3.6
        lca = ei2rmnd.multiregion_lca_without_double_counting(act_name, all_actvts, regions, eidb, demand)
        
        # bionames are only generated once. This takes some time.
        # probably these never change...
        if len(bio_names) != lca.inventory.shape[0]:
            print("Bionames changed, old length: {}".format(len(bio_names)))
            bio_names = [get_activity(key)["name"] for key in lca.biosphere_dict]
            print("new length: {}".format(len(bio_names)))

        # flows are aggregated by the resp. material
        # we do not care *where* the material flows to
        result[act_name] = pd.DataFrame.from_dict({
            "flow": bio_names,
            "amount": [lca.inventory[n, :].sum() for n in range(len(bio_names))]
        }).groupby("flow").agg({"amount": sum})

    df = pd.concat(result)
    
    inverse_techlookup = {
        itm: tech for tech 
        in eimod.available_electricity_generating_technologies 
        for itm in eimod.available_electricity_generating_technologies[tech]}
    df["RMND Tech"] = df.index.get_level_values(0).map(inverse_techlookup)
    
    return (df
            .reset_index()
            .drop("level_0", axis=1)
            .groupby(["RMND Tech", "flow"])
            .agg({"amount": sum}))

In [101]:
df = rmnd_bioflows_without_double_counting("EUR")

Found following regions for EUR: ['EUR', 'SE', 'FI', 'GR', 'GB', 'AX', 'HR', 'IT', 'ES', 'DE', 'PT', 'DK', 'FR', 'IE', 'BALTSO', 'FO', 'EE', 'NL', 'Canary Islands', 'CENTREL', 'CY', 'MT', 'IM', 'LT', 'AT', 'BE', 'BG', 'CZ', 'GI', 'HU', 'LU', 'LV', 'PL', 'RO', 'SI', 'SK', 'RER']
Processing activity Electricity, from CC plant, 100% SNG, truck 25km, post, pipeline 200km, storage 1000m/2025
Bionames changed, old length: 0
new length: 2079
Processing activity Electricity, at wood burning power plant 20 MW, truck 25km, post, pipeline 200km, storage 1000m/2025
Processing activity Electricity, at BIGCC power plant 450MW, pre, pipeline 200km, storage 1000m/2025
Processing activity Electricity, at BIGCC power plant 450MW, no CCS/2025
Processing activity Electricity, at power plant/hard coal, IGCC, no CCS/2025
Processing activity Electricity, at power plant/lignite, IGCC, no CCS/2025
Processing activity Electricity, at power plant/hard coal, pre, pipeline 200km, storage 1000m/2025
Processing acti

In [99]:
df.sample(20)

,,amount
RMND Tech,flow,
Coal PC CCS,"Transformation, to river, artificial",4.740223e-06
Solar CSP,Trimethylamine,6.339916e-14
Nuclear,Ethalfluralin,2.661067e-14
Coal PC CCS,"Volume occupied, underground deposit",6.547929e-08
Solar PV,"Transformation, from permanent crop, irrigated",8.830258e-08
Gas CC,"Occupation, mineral extraction site",8.007313e-06
Geothermal,"Copper, 0.99% in sulfide, Cu 0.36% and Mo 8.2E-3% in crude ore, in ground",1.353111e-06
Gas CC,Amine oxide,0.000000e+00
Solar PV,"Sodium nitrate, in ground",1.390420e-12


## Bioflows from the electricity sector with double counting

E.g., flows from wind power plants include coal-generated electricity impacts that are already being accounted for by a `electricity production, coal` lca.

In [105]:
def rmnd_bioflows_with_double_counting(region, scenario="BAU", year=2030, tech_primers={}):
    """ Extract bioflows for the REMIND electricity sector in a REMIND region.
        A dict with primers {x: y} can be given to specify ecoinvent technology y to represent
        REMIND technology x.
    """
    eidb_name = ei2rmnd.get_REMIND_database_name(scenario, year)
    eidb = Database(eidb_name)

    result = {}
    bio_names = []
    regions = ei2rmnd.ei_locations_in_remind_region(region)
    print("Found following regions for {}: {}".format(region, regions))
    for key in eimod.available_electricity_generating_technologies:
        print("Processing RMND Technology {}".format(key))
        if key in tech_primers:
            techs = [tech_primers[key]]
        else:
            techs = eimod.available_electricity_generating_technologies[key]
        if len(techs) == 0:
            print("No technologies available for {}! Skipping.".format(key))
            continue
        units = {
            "kilowatt hour": 1.,
            "megajoule": 1./3.6
        }
        lca = ei2rmnd.lca_for_multiple_techs_and_regions(
                techs, regions, eidb, units_and_conversions=units)
        
        # bionames are only generated once. This takes some time.
        # probably these never change...
        if len(bio_names) != lca.inventory.shape[0]:
            print("Bionames changed, old length: {}".format(len(bio_names)))
            bio_names = [get_activity(key)["name"] for key in lca.biosphere_dict]
            print("new length: {}".format(len(bio_names)))

        # flows are aggregated by the resp. material
        # we do not care *where* the material flows to
        result[key] = pd.DataFrame.from_dict({
            "flow": bio_names,
            "amount": [lca.inventory[n, :].sum() for n in range(len(bio_names))]
        }).groupby("flow").agg({"amount": sum})

    return pd.concat(result)

In [106]:
df_dc = rmnd_bioflows_with_double_counting("EUR")

Found following regions for EUR: ['EUR', 'SE', 'FI', 'GR', 'GB', 'AX', 'HR', 'IT', 'ES', 'DE', 'PT', 'DK', 'FR', 'IE', 'BALTSO', 'FO', 'EE', 'NL', 'Canary Islands', 'CENTREL', 'CY', 'MT', 'IM', 'LT', 'AT', 'BE', 'BG', 'CZ', 'GI', 'HU', 'LU', 'LV', 'PL', 'RO', 'SI', 'SK', 'RER']
Processing RMND Technology Biomass IGCC CCS
Bionames changed, old length: 0
new length: 2079
Processing RMND Technology Biomass IGCC
Processing RMND Technology Coal IGCC
Processing RMND Technology Coal IGCC CCS
Processing RMND Technology Coal PC CCS
Processing RMND Technology Gas CCS
Processing RMND Technology Biomass CHP
Processing RMND Technology Coal PC
Processing RMND Technology Coal CHP
Processing RMND Technology Gas OC
Processing RMND Technology Gas CC
Processing RMND Technology Gas CHP
Processing RMND Technology Geothermal
Processing RMND Technology Hydro
Processing RMND Technology Hydrogen
No technologies available for Hydrogen! Skipping.
Processing RMND Technology Nuclear
Processing RMND Technology Oil


In [110]:
df_dc.index = df_dc.index.rename(["RMND Tech", "flow"])

In [111]:
df_dc.sort_index()[:30]

amount
RMND Tech   flow                                               
Biomass CHP 1,3-Dioxolan-2-one                     5.504127e-10
            1,4-Butanediol                         6.419200e-12
            1-Pentanol                             6.018845e-13
            1-Pentene                              7.954006e-12
            2,2,4-Trimethyl pentane                4.339602e-15
            2,4-D                                  6.614032e-09
            2,4-D amines                           7.723629e-20
            2,4-D ester                            5.003715e-20
            2,4-DB                                 5.413703e-20
            2,4-di-tert-butylphenol                0.000000e+00
            2-Aminopropanol                        5.819360e-13
            2-Methyl pentane                       6.967736e-10
            2-Methyl-1-propanol                    1.777723e-12
            2-Methyl-2-butene                      3.223782e-16
            2-Nitrobenzoic acid                    3.030090e-13
            2-Propanol                             1.685840e-08
            2-chlorobenzaldehyde                   0.000000e+00
            3-Methyl-1-butanol                     0.000000e+00
            4-Methyl-2-pentanol                    1.236453e-19
            4-Methyl-2-pentanone                   1.708335e-12
            AOX, Adsorbable Organic Halogen as Cl  4.105488e-09
            Abamectin                              0.000000e+00
            Acenaphthene                           1.326423e-12
            Acenaphthylene                         1.224729e-13
            Acephate                               1.572855e-10
            Acetaldehyde                           3.569259e-07
            Acetamide                              2.311316e-11
            Acetamiprid                            0.000000e+00
            Acetic acid                            5.014071e-08
            Acetochlor                             1.259551e-13

## Impact of double counting

Compare versions with and without double counting.

In [120]:
fulldf = df.join(df_dc, lsuffix="_nodc", rsuffix="_dc")

In [121]:
fulldf["rel. (nodc/dc)"] = fulldf["amount_nodc"]/fulldf["amount_dc"]

In [122]:
fulldf["abs. (dc - nodc)"] = fulldf["amount_dc"] - fulldf["amount_nodc"]

In [123]:
fulldf.sort_values("abs. (dc - nodc)", ascending=False)[:30]

,,amount_nodc,amount_dc,rel. (nodc/dc),abs. (dc - nodc)
RMND Tech,flow,,,,
Biomass IGCC,Radon-222,0.000746,18.179998,0.000041,18.179252
Biomass IGCC CCS,Radon-222,0.001560,9.628468,0.000162,9.626908
Biomass IGCC,"Noble gases, radioactive, unspecified",0.000022,3.947591,0.000006,3.947569
Geothermal,Radon-222,0.000092,3.482510,0.000026,3.482418
Solar PV,Radon-222,0.001494,3.478273,0.000429,3.476779
Coal PC CCS,Radon-222,0.015446,2.533320,0.006097,2.517874
Coal IGCC CCS,Radon-222,0.007681,2.288813,0.003356,2.281131
Biomass IGCC CCS,"Noble gases, radioactive, unspecified",0.000040,2.105826,0.000019,2.105786
Coal PC,Radon-222,0.037656,2.074579,0.018151,2.036923
